# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 24 нояб

__Жесткий дедлайн: 27 нояб


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов, 2 балл за доп вопросы, если возникнут, если вопросов не возникло, считаем, что 2 балла вы получили 

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd
import re

pd.set_option('display.max_colwidth', 100000)

emails = pd.read_csv('emails.csv')
len(emails)

517401

In [2]:
emails.sample()

,file,message
490876,tycholiz-b/inbox/575.,"Message-ID: <24824667.1075861184658.JavaMail.evans@thyme>\nDate: Mon, 19 Nov 2001 12:40:57 -0800 (PST)\nFrom: karen.buckley@enron.com\nTo: mike.curry@enron.com, david.forster@enron.com, heather.kroll@enron.com, \n\tfred.lagrasta@enron.com, laura.luce@enron.com, jean.mrha@enron.com, \n\tbrian.redmond@enron.com, carl.tricoli@enron.com, \n\tbarry.tycholiz@enron.com, frank.vickers@enron.com\nSubject: Origination Track - Interviews Wednesday 28th.\nCc: louise.kitchen@enron.com, terry.peng@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nBcc: louise.kitchen@enron.com, terry.peng@enron.com\nX-From: Buckley, Karen </O=ENRON/OU=NA/CN=RECIPIENTS/CN=KBUCKLEY>\nX-To: Curry, Mike </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Mcurry>, Forster, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=DFORSTER>, Kroll, Heather </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Hkroll>, Lagrasta, Fred </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Flagras>, Luce, Laura </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Lluce>, Mrha, Jean </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Jmrha>, Redmond, Brian </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Bredmon>, Tricoli, Carl </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Ctricol>, Tycholiz, Barry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Btychol>, Vickers, Frank </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Fvicker>\nX-cc: Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Lkitchen>, Peng, Terry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Tpeng>\nX-bcc: \nX-Folder: \BTYCHOL (Non-Privileged)\Tycholiz, Barry\Inbox\nX-Origin: Tycholiz-B\nX-FileName: BTYCHOL (Non-Privileged).pst\n\nAll,\n \n \nBelow are the confirmed list of names to be interviewed for the Origination Track, Wednesday, 28th November. (there are a possibility of a further 3 candidates).\n \nAll of you are confirmed to interview, however I require another 4/5 interviewers (Manager/director level) to ensure all candidates are interviewed by 5 originators. Pls advise who from your groups can facilitate interviewing.\n \nInterview schedules/resumes will be sent to you early next week.\n \nI look forward to hearing from you. \n \nRegards,\n \nKaren Buckley<?xml:namespace prefix = o ns = ""urn:schemas-microsoft-com:office:office"" />\n\nHuman Resources\n\nTel: 713 345 4667\n\n \n \n \nFIRST\t LAST\t EXTENSION\t MANAGER\t GROUP\t LEVEL\t\nAdam\t Bayer\t 35227\t Christopher Gaskill\t ENA Fundamental Analysis\t Analyst\t\nHarry\t Bucalo\t 39196\t Mark Russ\t EBS Voice minutes\t Associate\t\nBart\t Burk\t 53635\t Christopher Gaskill\t ENA Fundamental Analysis\t Analyst\t\nCatalina\t Cardenas\t 53650\t Christopher Gaskill\t ENA Fundamental Analysis\t Analyst\t\nMarcus\t Edmonds\t 34243\t Lisa Bills\t ENA Treasury & Funding\t Analyst\t\nAndrea \tGonzalez\t 58128\t Carl Tricoli\t ENA Generation Investments\t Associate\t\nRyan \tHinze\t 58358\t Christopher Higert\t ENA Compression Services\t Analyst\t\nDaniel\t Jenkins\t 53398\t Rika Imai\t ENA Fundamental Analysis\t Analyst\t\nJohn\t Lang\t 39947\t Charles Vetters\t ENA Generation Investments\t Analyst\t\nJimmy\t Marks\t 53656\t Mitchell Robinson\t ENA Generation Investments\t Analyst\t\nMaurico\t Marquez\t 36161\t Rika Imai\t ENA Fundamental Analysis\t Associate\t\nMichael \tOlsen\t 35796\t Jerry Farmer\t ENW Energy Ops Texas Logistics\t Analyst\t\nJustin\t O'Malley\t 58934\t Christopher Gaskill\t ENA Fundamental Analysis\t Analyst\t\nLauren\t Schlesinger\t 53268\t Christopher Gaskill\t ENA Fundamental Analysis\t Analyst\t\nKyle\t Simson\t 39950\t Christopher Harman\t EBS Trading Real Estate\t Sr Specialist\t\n\n \n\n \n\n \n\n \n\n \n\n"


Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [3]:
# достали везде HTTP Body
emails['message'] = emails.message.str.extract(r"\n\n(.*)", flags=re.DOTALL)

In [4]:
emails['message'].replace(
    {
        r"(?i)(from|to|subject|date|sent|mime-version|content-type|cc):.*\n": "",
        r"[-=]+.*\n": "",
        r'<.*?>': "",
        r"[*]+": "",
        r'\s+': " ",
        r'https?://\S+|www\.\S+': "",
        r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}': "",
        r'[^\w\s]': ""
    }, 
    inplace=True,
    regex=True
)

/var/folders/6w/m0f3kdj936z8dc656nxnyt_c0000gq/T/ipykernel_98613/211835742.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  emails['message'].replace(


In [5]:
emails.sample(10)

file  \
9901            bailey-s/deleted_items/89.   
300358             mann-k/_sent_mail/1883.   
373401  rodrique-r/discussion_threads/121.   
485414                  taylor-m/sent/485.   
464730         symes-k/all_documents/2934.   
376150        rogers-b/all_documents/1658.   
67481        dasovich-j/deleted_items/131.   
35368                blair-l/meetings/917.   
345783         nemec-g/all_documents/4424.   
339640   mims-thurston-p/deleted_items/98.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [6]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        for word in vocabulary:
            self._insert(word)

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        node = self._find_node(prefix)

        if node is None:
            return []
        
        words = []
        self._find_words(node, prefix, words)
        return words

    def _insert(self, word: str):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTreeNode()
            node = node.children[char]
        node.is_end_of_word = True

    def _find_node(self, prefix: str) -> PrefixTreeNode:
        node = self.root
        for char in prefix:
            if char not in node.children:
                return None
            node = node.children[char]
        return node

    def _find_words(self, node: PrefixTreeNode, prefix: str, words: List[str]):
        if node.is_end_of_word:
            words.append(prefix)

        for char, child_node in node.children.items():
            self._find_words(child_node, prefix + char, words)

In [7]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [8]:
import numpy as np

class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        words = pd.Series(corpus).explode().tolist()
        self.probabilties = dict(zip(*np.unique(words, return_counts=True)))
        self.total = sum([value for value in self.probabilties.values()])
        self.prefix_tree = PrefixTree(words)

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """

        words = self.prefix_tree.search_prefix(prefix)
        probs = [self.probabilties.get(word) / self.total for word in words if word in self.probabilties]
        
        return words, probs

In [9]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [10]:
from collections import defaultdict

class NGramLanguageModel:
    def __init__(self, corpus, n):
        self.n = n
        self.ngrams = defaultdict(list)

        self._initialize_model(corpus)

    def _initialize_model(self, corpus):
        for sentence in corpus:
            sent_len = len(sentence)
            for i in range(sent_len):
                for j in range(1, sent_len - i + 1):
                    prefix = tuple(sentence[i:i + j - 1])
                    next_word = sentence[i + j - 1]
                    self.ngrams[prefix].append(next_word)

    def get_next_words_and_probs(self, prefix: List[str]) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после prefix, 
        а также список вероятностей этих слов.
        """
        next_words = self.ngrams.get(tuple(prefix), [])
        next_words_len = len(next_words)

        words = [word for word in next_words]
        probs = [next_words.count(word) / next_words_len for word in next_words]

        return words, probs


In [11]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [12]:
from typing import Union

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        words = text.strip().split() if isinstance(text, str) else text[:]
        if not words:
            return []

        # Завершаем последнее слово, если оно неполное
        last_word = words[-1]
        completions, probs = self.word_completor.get_words_and_probs(last_word)
        completed_word = max(zip(completions, probs), key=lambda x: x[1], default=(last_word, 0))[0]

        # Обновляем список слов
        full_words = words[:-1] + [completed_word]
        context = full_words[-(self.n_gram_model.n - 1):]

        # Генерация продолжений
        suggestion = [completed_word]
        for _ in range(n_words):
            next_words, next_probs = self.n_gram_model.get_next_words_and_probs(context)
            if not next_words:
                break
            next_word = max(zip(next_words, next_probs), key=lambda x: x[1])[0]
            suggestion.append(next_word)
            context = context[1:] + [next_word]  # Обновляем контекст

        return [suggestion]  # Возвращаем только один вариант продолжения

In [13]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [14]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

In [22]:
emails.sample(100)['message'].astype(str).to_csv("handler_emails_100.csv")

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!